In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from numpy.linalg import inv
from sklearn import preprocessing

In [48]:
df = pd.read_csv('train_data.csv')
df

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,1565607600,30.329370,59.940488,63,4,52,6,0,0,0,0
1,1565568000,30.342845,59.869542,44,0,40,0,4,0,0,0
2,1565568000,30.322632,60.039332,30,0,77,7,0,0,0,1
3,1565568000,30.322632,59.938238,13,0,634,17,32,0,0,7
4,1565568000,30.309158,59.932612,8,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8526521,1565978400,30.324878,59.932612,48,0,15,2,0,0,0,1
8526522,1565978400,30.324878,59.932612,20,1,0,0,0,0,0,0
8526523,1565978400,30.324878,59.932612,54,2,75,7,0,0,0,6
8526524,1565971200,30.324878,59.932612,15,0,350,4,25,0,0,0


In [49]:
sorted(df.lat.unique())

[-1.3900065324650763,
 0.0,
 35.032142207354845,
 41.30376021953485,
 42.289870330403645,
 42.99689995244231,
 43.68939181898115,
 52.11667860590217,
 54.71500653820782,
 54.77463685891882,
 55.70797485949529,
 55.72189070179645,
 55.75855416422385,
 56.9590331018085,
 57.45166050067814,
 59.656929822910136,
 59.71474042375376,
 59.74756864368603,
 59.75888124818248,
 59.80635237279253,
 59.80748181472322,
 59.8187741296651,
 59.81990315074165,
 59.82103213356388,
 59.82328998444886,
 59.82441885251328,
 59.82893394227072,
 59.83006261908929,
 59.83119125766129,
 59.83231985798753,
 59.833448420068855,
 59.8345769439062,
 59.83570542950036,
 59.83683387685223,
 59.83796228596263,
 59.83909065683245,
 59.840218989462535,
 59.84134728385376,
 59.84247554000695,
 59.84360375792299,
 59.84473193760274,
 59.84586007904705,
 59.84698818225677,
 59.84811624723277,
 59.84924427397591,
 59.85037226248704,
 59.85150021276701,
 59.852628124816725,
 59.85375599863699,
 59.85488383422868,
 59.85601

In [50]:
sorted(df.lon.unique())

[-48.41021066120101,
 0.0,
 18.84665466082756,
 20.486080054345685,
 23.69081983044208,
 24.000738603463308,
 29.40635082565253,
 30.08907044158337,
 30.091316229793676,
 30.09356201800397,
 30.09580780621427,
 30.098053594424574,
 30.100299382634866,
 30.102545170845165,
 30.104790959055453,
 30.107036747265763,
 30.10928253547606,
 30.11152832368636,
 30.113774111896657,
 30.11601990010696,
 30.120511476527557,
 30.12275726473785,
 30.125003052948152,
 30.127248841158448,
 30.12949462936875,
 30.13174041757905,
 30.13398620578934,
 30.136231993999644,
 30.138477782209943,
 30.140723570420235,
 30.142969358630538,
 30.14521514684084,
 30.14746093505114,
 30.14970672326144,
 30.15195251147173,
 30.15419829968204,
 30.15644408789233,
 30.158689876102628,
 30.16093566431293,
 30.163181452523236,
 30.16542724073353,
 30.16767302894383,
 30.16991881715413,
 30.17216460536443,
 30.174410393574725,
 30.176656181785024,
 30.178901969995323,
 30.181147758205626,
 30.18339354641592,
 30.1856393

In [51]:
# нижняя и верхняя граница по широте должна быть 59.656929822910136 и 60.374089897957774
# по долготе 30.08907044158337 и 30.61009330637269
true_lon = sorted(df.lon.unique()[(df.lon.unique() >= 30.08907044158337) & (df.lon.unique() <= 30.61009330637269)])
true_lon

[30.08907044158337,
 30.091316229793676,
 30.09356201800397,
 30.09580780621427,
 30.098053594424574,
 30.100299382634866,
 30.102545170845165,
 30.104790959055453,
 30.107036747265763,
 30.10928253547606,
 30.11152832368636,
 30.113774111896657,
 30.11601990010696,
 30.120511476527557,
 30.12275726473785,
 30.125003052948152,
 30.127248841158448,
 30.12949462936875,
 30.13174041757905,
 30.13398620578934,
 30.136231993999644,
 30.138477782209943,
 30.140723570420235,
 30.142969358630538,
 30.14521514684084,
 30.14746093505114,
 30.14970672326144,
 30.15195251147173,
 30.15419829968204,
 30.15644408789233,
 30.158689876102628,
 30.16093566431293,
 30.163181452523236,
 30.16542724073353,
 30.16767302894383,
 30.16991881715413,
 30.17216460536443,
 30.174410393574725,
 30.176656181785024,
 30.178901969995323,
 30.181147758205626,
 30.18339354641592,
 30.185639334626217,
 30.18788512283652,
 30.19013091104681,
 30.19237669925711,
 30.19462248746741,
 30.19686827567771,
 30.19911406388801,

In [57]:
true_lat = sorted(df.lat.unique()[(df.lat.unique() >= 59.656929822910136) & (df.lat.unique() <= 60.374089897957774)])
true_lat

[59.656929822910136,
 59.71474042375376,
 59.74756864368603,
 59.75888124818248,
 59.80635237279253,
 59.80748181472322,
 59.8187741296651,
 59.81990315074165,
 59.82103213356388,
 59.82328998444886,
 59.82441885251328,
 59.82893394227072,
 59.83006261908929,
 59.83119125766129,
 59.83231985798753,
 59.833448420068855,
 59.8345769439062,
 59.83570542950036,
 59.83683387685223,
 59.83796228596263,
 59.83909065683245,
 59.840218989462535,
 59.84134728385376,
 59.84247554000695,
 59.84360375792299,
 59.84473193760274,
 59.84586007904705,
 59.84698818225677,
 59.84811624723277,
 59.84924427397591,
 59.85037226248704,
 59.85150021276701,
 59.852628124816725,
 59.85375599863699,
 59.85488383422868,
 59.85601163159266,
 59.85713939072978,
 59.8582671116409,
 59.85939479432693,
 59.86052243878864,
 59.86165004502693,
 59.86277761304268,
 59.86390514283672,
 59.86503263440992,
 59.86616008776313,
 59.86728750289723,
 59.86841487981305,
 59.8695422185115,
 59.87066951899336,
 59.87179678125955,


In [53]:
# Оставим в датасете только интересующие нас долготы и широты и переведем timestamp в даты
df['timestamp']= [datetime.fromtimestamp(x) for x in df['timestamp']]
df_clear = df[(df.lat >= true_lat[0]) & (df.lat <= true_lat[-1])]
df_clear = df[(df.lon >= true_lon[0]) & (df.lon <= true_lon[-1])]
df_clear

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt
0,2019-08-12 14:00:00,30.329370,59.940488,63,4,52,6,0,0,0,0
1,2019-08-12 03:00:00,30.342845,59.869542,44,0,40,0,4,0,0,0
2,2019-08-12 03:00:00,30.322632,60.039332,30,0,77,7,0,0,0,1
3,2019-08-12 03:00:00,30.322632,59.938238,13,0,634,17,32,0,0,7
4,2019-08-12 03:00:00,30.309158,59.932612,8,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8526521,2019-08-16 21:00:00,30.324878,59.932612,48,0,15,2,0,0,0,1
8526522,2019-08-16 21:00:00,30.324878,59.932612,20,1,0,0,0,0,0,0
8526523,2019-08-16 21:00:00,30.324878,59.932612,54,2,75,7,0,0,0,6
8526524,2019-08-16 19:00:00,30.324878,59.932612,15,0,350,4,25,0,0,0


In [58]:
true_lat.reverse()
true_lat

[60.374089897957774,
 60.1792298512455,
 60.11662921549629,
 60.07296190858619,
 60.06063502425536,
 60.05951417000399,
 60.05503037228068,
 60.05278824497618,
 60.04942476844644,
 60.04269678723865,
 60.04157532374879,
 60.04045382217418,
 60.03933228251395,
 60.03821070476726,
 60.03708908893322,
 60.035967435010974,
 60.03484574299966,
 60.033724012898425,
 60.03260224470638,
 60.03148043842269,
 60.03035859404648,
 60.02923671157687,
 60.02811479101304,
 60.02699283235407,
 60.025870835599136,
 60.02474880074738,
 60.0236267277979,
 60.022504616749856,
 60.02138246760239,
 60.020260280354655,
 60.01913805500574,
 60.01801579155483,
 60.01689349000102,
 60.01577115034348,
 60.01464877258133,
 60.013526356713705,
 60.01240390273974,
 60.0112814106586,
 60.010158880469376,
 60.00903631217126,
 60.00791370576335,
 60.00679106124481,
 60.00566837861474,
 60.004545657872285,
 60.00342289901664,
 60.00230010204685,
 60.00117726696214,
 60.00005439376158,
 59.99893148244436,
 59.9978085330

In [ ]:
# добавим сетку в наш датасет

# y = 0
# for latitude in tqdm(true_lat):
#     x = 0
#     for longitude in true_lon:
#         for index, row in df_clear.iterrows():
#             if (row.lat == latitude) & (row.lon == longitude):
#                 df_clear['x'] = x
#                 df_clear['y'] = y
#         x += 1
#     y += 1

In [14]:
df_clear["x"] = np.nan
df_clear['y'] = np.nan
df_clear

C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\597042464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear["x"] = np.nan
C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\597042464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear['y'] = np.nan


,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,x,y
0,2019-08-12 14:00:00,30.329370,59.940488,63,4,52,6,0,0,0,0,NaN,NaN
1,2019-08-12 03:00:00,30.342845,59.869542,44,0,40,0,4,0,0,0,NaN,NaN
2,2019-08-12 03:00:00,30.322632,60.039332,30,0,77,7,0,0,0,1,NaN,NaN
3,2019-08-12 03:00:00,30.322632,59.938238,13,0,634,17,32,0,0,7,NaN,NaN
4,2019-08-12 03:00:00,30.309158,59.932612,8,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8526521,2019-08-16 21:00:00,30.324878,59.932612,48,0,15,2,0,0,0,1,NaN,NaN
8526522,2019-08-16 21:00:00,30.324878,59.932612,20,1,0,0,0,0,0,0,NaN,NaN
8526523,2019-08-16 21:00:00,30.324878,59.932612,54,2,75,7,0,0,0,6,NaN,NaN
8526524,2019-08-16 19:00:00,30.324878,59.932612,15,0,350,4,25,0,0,0,NaN,NaN


In [38]:
flag = False
df_clear["x"] = np.nan
df_clear['y'] = np.nan
for index, row in tqdm(df_clear.iterrows()):
    flag = False
    if (index+1) % 100 == 0:
        save_df = df_clear
        save_df.to_csv('clear_data.csv', encoding='utf-8')
    for y, latitude in enumerate(true_lat):
        for x, longitude in enumerate(true_lon):
            if (row.lat == latitude) & (row.lon == longitude):
                df_clear['x'].iloc[index] = x
                df_clear['y'].iloc[index] = y
                flag = True
                break
        if flag:
            break


C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\444733394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear["x"] = np.nan
C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\444733394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear['y'] = np.nan
0it [00:00, ?it/s]C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\444733394.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

KeyboardInterrupt: 

In [43]:
pd.read_csv('clear_data.csv')

,Unnamed: 0,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,x,y,-2,-1
0,0,2019-08-12 14:00:00,30.329370,59.940488,63,4,52,6,0,0,0,0,106.0,100.0,106.0,100.0
1,1,2019-08-12 03:00:00,30.342845,59.869542,44,0,40,0,4,0,0,0,112.0,163.0,112.0,163.0
2,2,2019-08-12 03:00:00,30.322632,60.039332,30,0,77,7,0,0,0,1,103.0,12.0,103.0,12.0
3,3,2019-08-12 03:00:00,30.322632,59.938238,13,0,634,17,32,0,0,7,103.0,102.0,103.0,102.0
4,4,2019-08-12 03:00:00,30.309158,59.932612,8,0,0,0,0,0,0,0,97.0,107.0,97.0,107.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862475,1880240,2019-01-06 18:00:00,30.250767,59.932612,74,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1862476,1880241,2019-01-06 18:00:00,30.372040,59.965226,74,9,98,10,0,0,0,2,NaN,NaN,NaN,NaN
1862477,1880242,2019-01-06 18:00:00,30.342845,59.926986,58,8,58,6,0,0,0,2,NaN,NaN,NaN,NaN
1862478,1880243,2019-01-06 18:00:00,30.266488,60.006791,27,0,32,4,0,0,0,1,NaN,NaN,NaN,NaN


In [62]:
le = preprocessing.LabelEncoder()
le.fit(true_lon)
df_clear['x'] = le.transform(df_clear.lon)
le.fit(true_lat)
df_clear['y'] = le.transform(df_clear.lat)

C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\3356847411.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear['x'] = le.transform(df_clear.lon)
C:\Users\gto_n\AppData\Local\Temp\ipykernel_10388\3356847411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clear['y'] = le.transform(df_clear.lat)


In [64]:
df_clear

,timestamp,lon,lat,likescount,commentscount,symbols_cnt,words_cnt,hashtags_cnt,mentions_cnt,links_cnt,emoji_cnt,x,y
0,2019-08-12 14:00:00,30.329370,59.940488,63,4,52,6,0,0,0,0,106,110
1,2019-08-12 03:00:00,30.342845,59.869542,44,0,40,0,4,0,0,0,112,47
2,2019-08-12 03:00:00,30.322632,60.039332,30,0,77,7,0,0,0,1,103,198
3,2019-08-12 03:00:00,30.322632,59.938238,13,0,634,17,32,0,0,7,103,108
4,2019-08-12 03:00:00,30.309158,59.932612,8,0,0,0,0,0,0,0,97,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8526521,2019-08-16 21:00:00,30.324878,59.932612,48,0,15,2,0,0,0,1,104,103
8526522,2019-08-16 21:00:00,30.324878,59.932612,20,1,0,0,0,0,0,0,104,103
8526523,2019-08-16 21:00:00,30.324878,59.932612,54,2,75,7,0,0,0,6,104,103
8526524,2019-08-16 19:00:00,30.324878,59.932612,15,0,350,4,25,0,0,0,104,103
